DEAP (Distributed Evolutionary Algorithms in Python) es un framework para la implementación de algoritmos evolutivos en Python. Trabaja fácilmente con paralelización y multiprocesamiento. Además, nos brinda un conjunto de herramientas que rápidamente nos permite implementar y testear algoritmos evolutivos.

https://deap.readthedocs.io/en/master/

In [ ]:
 !pip install deap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Lo primero que haremos es importar los módulos necesarios para que el código pueda funcionar.

1. Generación del dataset y creación de la función de evaluación.
2. Instanciación del toolbox de DEAP y configuración del problema y los operadores.
3. Generación de un Algoritmo Genético Simple.

In [ ]:
from deap import base
from deap import creator
from deap import tools
from deap import algorithms
from deap import benchmarks
import random
import numpy

Es creator es una fábrica de clases que puede crear nuevas clases en tiempo de ejecución. 

La clase Fitness provee es una clase abstracta que requiere un argumento weights, esta clase me permite crear un problema de maximización (1.0,) o minimización (-1.0,), tambien puedo crear problemas multiobjetivo (1.0,-1.0,) donde se buscará por ejemplo maximizar la primera función objetivo y minimizar la segunda función objetivo. 


* No omitir la coma (,) al final del weights de evaluación, esta es requerida por el framework

los diferentes tipos de algoritmos evolutivos (GA, GP, ES, PSO, DE, …) utiliza diferente representación para los individuos, en este caso Algoritmo Genetico Básico utilizaremos una lista para representar los individuos.


In [ ]:
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

/usr/local/lib/python3.9/dist-packages/deap/creator.py:138: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/usr/local/lib/python3.9/dist-packages/deap/creator.py:138: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


Se utilizan clases personalizadas para crear tipos que representen a nuestros individuos, así como a toda nuestra población.

In [ ]:
toolbox = base.Toolbox()
#Se utiliza un generador de numeros aleatorios (numeros enteros) para generar los alelos entre -30 y 30
toolbox.register("attr_int", random.randint, 0, 64)
# Se crea un individuo de longitud 16 utilizando el generador de alelos anterior
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_int, 8)
# Se crea la población como una lista de individuos utilizando el creador de individuos anterior
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

En este caso configuraremos un algoritmo genético para encontrar el máximo de la función de ackley, 
Deap proporciona unas funciones de prueba disponibles en:

https://deap.readthedocs.io/en/master/api/benchmarks.html


Usted puede programar su propia función de evaluación.


def evalOneMax(individual):
    return sum(individual),

* No omitir la coma (,) al final de la función de evaluación, esta es requerida por el framework

In [ ]:
toolbox.register("evaluate",benchmarks.schaffer)

In [ ]:

toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.1)
toolbox.register("select", tools.selTournament, tournsize=3)

El toolbox módulo contiene las implementación de los operadores selección, cruce y mutación 
https://deap.readthedocs.io/en/master/api/tools.html

In [ ]:
def main():
    random.seed(64)

    # Tamaño de la población
    pop = toolbox.population(n=300)
    #Probabilidad de cruce y mutación
    CXPB, MUTPB = 0.5, 0.2
    # Número de generaciones
    NGEN=50

    # Estructura que almacena el mejor individuo
    hof = tools.HallOfFame(1)

    # Se define las estádisticas que se obtendran por generación
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", numpy.mean)
    stats.register("std", numpy.std)
    stats.register("min", numpy.min)
    stats.register("max", numpy.max)

    #Se crea el algoritmo genetiso simple utilizando la configuración anterior

    pop, log = algorithms.eaSimple(pop, toolbox, cxpb=CXPB, mutpb=MUTPB, ngen=NGEN, 
                                   stats=stats, halloffame=hof, verbose=True)
    return pop, stats, hof 

if __name__ == "__main__":
    #Ejecución del Algoritmo Genetico
    p,s,h=main()
    #Impresión de la solución encontrada
    print(h)

gen	nevals	avg    	std    	min   	max    
0  	300   	73.2349	11.0177	39.142	101.797
1  	156   	81.3044	8.63398	56.7343	112.434
2  	186   	85.9444	9.05446	49.6081	112.434
3  	177   	91.1093	8.76147	53.6156	112.434
4  	199   	95.3517	9.11839	61.6907	112.913
5  	169   	99.2806	9.28897	44.5686	115.932
6  	170   	103.274	6.93592	71.0563	117.463
7  	185   	106.39 	6.4896 	80.1147	119.503
8  	182   	108.517	6.96495	73.6453	119.503
9  	172   	110.319	7.38974	58.1347	120.629
10 	182   	112.78 	6.3185 	74.3507	121.165
11 	183   	114.507	5.82691	76.6406	123.614
12 	194   	115.476	6.81282	65.1748	123.614
13 	209   	116.596	7.80632	58.1763	123.614
14 	162   	118.432	6.8451 	73.7286	124.506
15 	182   	119.612	7.32484	70.3514	125.466
16 	186   	120.176	6.90515	80.0665	125.466
17 	161   	121.093	6.41436	79.6264	125.466
18 	178   	121.407	6.5013 	79.2227	125.466
19 	160   	121.831	6.83911	79.2227	125.466
20 	202   	121.414	7.84077	79.273 	125.466
21 	190   	122.677	6.45099	82.7773	125.466
22 	191   	12